In [66]:
from pymongo import MongoClient

client = MongoClient('129.150.114.173', 27017)
db = client.TwitterStream;

Topics = db["topic"];
print(Topics);


#Get all the Tweets from Database-----------------
Tweets = [];
for obj in Topics.find({}):
    Tweets.append(obj);

Collection(Database(MongoClient(host=['129.150.114.173:27017'], document_class=dict, tz_aware=False, connect=True), 'TwitterStream'), 'topic')


In [69]:
#objects2 = Tweets[:];

import json
import re
import datetime

#Clean the Tweets or Preprocessing--------------------------

tweets_cleaned = [] 
#tweet_content = db.tweet.find()

#print(u"\U0001F600-\U0001F64F")
emoji_pattern = re.compile(
    u"(\ud83d[\ude00-\ude4f])|"  # emoticons
    u"(\ud83c[\udf00-\uffff])|"  # symbols & pictographs (1 of 2)
    u"(\ud83d[\u0000-\uddff])|"  # symbols & pictographs (2 of 2)
    u"(\ud83d[\ude80-\udeff])|"  # transport & map symbols
    u"(\ud83c[\udde0-\uddff])"  # flags (iOS)
    "+", flags=re.UNICODE)#emoji_pattern = re.compile('\S')
hashtag_pattern = re.compile('\#')#('\#\S+[\s|$]')
mention_pattern = re.compile('\@\S+\s|$')
#adding the keywords to remove the obivous noice
bad_words = []

for tweet in Tweets:

    tweet['text'] = tweet['text'].lower() #change all the character into lowercase
    tweet['text'] = re.sub(r'http\S+','',tweet['text'])#remove url
    tweet['text'] = mention_pattern.sub(r'',tweet['text'])#remove @username
    #tweet['text'] = re.sub(r'&amp','',tweet['text'])#remove retweet
    #tweet['text'] = hashtag_pattern.sub(r'',tweet['text'])#remove hashtag
    tweet['text'] = emoji_pattern.sub(r'',tweet['text'])#remove emoji
    if not any(bad_word in tweet['text'] for bad_word in bad_words):
        #print(tweet['text'])#remove obivous noice
        tweets_cleaned.append(tweet);
        #if you want to save back into mongodb, define another collection first  
        #print(tweet)


In [70]:
#------------------ Apply the LDA Model --------------------

from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc['text']).split() for doc in tweets_cleaned]


# Importing Gensim
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

### Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel
# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word = dictionary, passes=50)



#--------------------- Get all the topics ------------------------
topics = ldamodel.print_topics(num_topics=10, num_words=2);
print(topics);


[(0, '0.099*"trump" + 0.064*"rt"'), (1, '0.043*"rt" + 0.019*"sick"'), (2, '0.031*"rt" + 0.027*"football"'), (3, '0.048*"rt" + 0.015*"book"'), (4, '0.067*"rt" + 0.061*"food"'), (5, '0.048*"rt" + 0.014*"trump"'), (6, '0.048*"rt" + 0.038*"trump"'), (7, '0.060*"rt" + 0.026*"retweet"'), (8, '0.067*"trump" + 0.062*"rt"'), (9, '0.072*"trump" + 0.054*"rt"')]


In [74]:
#------- Associate each Tweet a Topic by its maximmum probability ------

import numpy as np
#get topics for each document
topics_documents = np.zeros(len(tweets_cleaned));

for i in range(len(objects2)):
    bow = dictionary.doc2bow(tweets_cleaned[i]['text'].split())
    #get the maximmum topic probability of each document
    distribution_topic = ldamodel.get_document_topics(bow);
    topics_documents[i] = distribution_topic.index(max(distribution_topic));

print(topics_documents[1:50]);

[ 9.  3.  3.  9.  9.  4.  9.  4.  9.  9.  9.  6.  9.  9.  9.  6.  9.  3.
  2.  5.  9.  9.  4.  9.  9.  1.  4.  4.  0.  4.  9.  6.  9.  9.  9.  4.
  9.  5.  9.  6.  3.  3.  3.  9.  9.  4.  4.  2.  3.]


In [126]:
#----------- Final Data Wanted ----------
#Dimensions: Topic - Number of Tweets - Day/Hour

#data.groupby(['month', 'item'])['date'].count()

import pandas as pd
from datetime import date
import dateutil

data_frame = pd.DataFrame(tweets_cleaned);
data_frame['TopicModelled'] = topics_documents;
#data_frame['created'] = data_frame['created'].apply(datetime.datetime.strptime, args=('%A %B %d %H:%M:%S +0000 %Y',))
data_frame['created'] = data_frame['created'].apply(dateutil.parser.parse)
#data_frame['created'] = data_frame['created'].apply(date.strftime("%d/%m/%y"))

#Choose the right grouping of date by
#%B - Month; %d - day; %H - Hour 
format_date_group = '%B %d %H';
aggregated_data = data_frame.groupby([data_frame['created'].dt.strftime(format_date_group), 'TopicModelled'])['TopicModelled'].count();

#----------- Data Frame with all the data needed --------------
aggregated_data

created      TopicModelled
April 10 19  0.0                29
             1.0                57
             2.0               364
             3.0               562
             4.0               698
             5.0               470
             6.0               293
             7.0               106
             8.0                19
             9.0              1969
April 10 20  0.0                26
             1.0                82
             2.0               571
             3.0               998
             4.0              1408
             5.0              1012
             6.0               694
             7.0               273
             8.0                76
             9.0              4435
April 11 19  1.0                 3
             2.0                61
             3.0               171
             4.0               375
             5.0               417
             6.0               423
             7.0               107
             8.0            